In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 25.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(temperature=0)

chain_input = {"equation_statement": RunnablePassthrough()}
runnable = (
    chain_input | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION: 
Subtract 7 from both sides:
x^3 = 5

Take the cube root of both sides:
x = ∛5


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

chain_input = {"context": retriever, "question": RunnablePassthrough()}
retrieval_chain = (
    chain_input
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [8]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

chain_input = RunnablePassthrough.assign(
    **{
        "context": itemgetter('question') | retriever, 
        "question": itemgetter('question'),
    }
)
retrieval_chain = (
    chain_input
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [11]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

chain_input = RunnablePassthrough.assign(
    context = itemgetter('question') | retriever).assign(
    question = itemgetter('question')
)
retrieval_chain = (
    chain_input
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [42]:
from operator import itemgetter

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.vectorstores import FAISS
from langchain.callbacks.tracers import ConsoleCallbackHandler

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

chat_history = ChatMessageHistory()

model = ChatOpenAI()

system_template = """You are a helpful chatbot. 

Your task is to answer questions given a context and a chat history.

Follow the instructions below.
% INSTRUCTIONS:
- only answer the questions using the context or chat history below
- if you don't know the answer, just answer: 'I don't know'

% CONTEXT:
{context}

% CHAT HISTORY:
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "\n% QUESTION:\n{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True, chat_memory=chat_history)

print(memory.load_memory_variables({}))
# {'history': [HumanMessage(content='hi im bob'), AIMessage(content='Hello Bob! How can I assist you today?')]}

chain_input =  RunnablePassthrough.assign(
    history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")).assign(
    context = itemgetter('input') | retriever
)
chain =  chain_input | prompt | model

{'history': []}


In [43]:
inputs = {"input": "what is my name"}
response = chain.invoke(inputs, config={'callbacks': [ConsoleCallbackHandler()]})
print(response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history>] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history>] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/end] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] s] Exiting Chain 

In [44]:
chat_history.add_user_message("hi im harison")
chat_history.add_ai_message('Hello harison! How can I assist you today?')
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='hi im harison'), AIMessage(content='Hello harison! How can I assist you today?')]}


In [45]:
# questions
inputs = {"input": "what is my name"}
response = chain.invoke(inputs, config={'callbacks': [ConsoleCallbackHandler()]})
print(response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history>] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history>] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] Entering Chain run with input:
{
  "input": "what is my name"
}
[chain/end] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] s] Exiting Chain 

In [46]:
# questions
inputs = {"input": "where harrison work"}
response = chain.invoke(inputs, config={'callbacks': [ConsoleCallbackHandler()]})
print(response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "where harrison work"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history>] Entering Chain run with input:
{
  "input": "where harrison work"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history>] Entering Chain run with input:
{
  "input": "where harrison work"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "where harrison work"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] Entering Chain run with input:
{
  "input": "where harrison work"
}
[chain/end] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variable

In [47]:
chat_history.add_user_message("what is langchain")
chat_history.add_ai_message('LangChain is an open-source framework that helps developers build applications using large language models (LLMs)')
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='hi im harison'), AIMessage(content='Hello harison! How can I assist you today?'), HumanMessage(content='what is langchain'), AIMessage(content='LangChain is an open-source framework that helps developers build applications using large language models (LLMs)')]}


In [49]:
inputs = {"input": "Is langchain open-source or closed-source?"}
response = chain.invoke(inputs, config={'callbacks': [ConsoleCallbackHandler()]})
print(response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Is langchain open-source or closed-source?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history>] Entering Chain run with input:
{
  "input": "Is langchain open-source or closed-source?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history>] Entering Chain run with input:
{
  "input": "Is langchain open-source or closed-source?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Is langchain open-source or closed-source?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] Entering Chain run with input:
{
  "input": "Is langchain open-source or closed-source?"
}
[chain/end] [chain:RunnableSequence > ch

In [50]:
chat_history.add_user_message("what is linux")
chat_history.add_ai_message("Linux is an open-source operating system (OS) that manages a computer's hardware and resources, like its CPU, memory.")
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='hi im harison'), AIMessage(content='Hello harison! How can I assist you today?'), HumanMessage(content='what is langchain'), AIMessage(content='LangChain is an open-source framework that helps developers build applications using large language models (LLMs)'), HumanMessage(content='what is linux'), AIMessage(content="Linux is an open-source operating system (OS) that manages a computer's hardware and resources, like its CPU, memory.")]}


In [51]:
inputs = {"input": "What is Linux?"}
response = chain.invoke(inputs, config={'callbacks': [ConsoleCallbackHandler()]})
print(response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is Linux?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history>] Entering Chain run with input:
{
  "input": "What is Linux?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history>] Entering Chain run with input:
{
  "input": "What is Linux?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is Linux?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] Entering Chain run with input:
{
  "input": "What is Linux?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableAssign<history> > chain:RunnableParallel<history> > chain:RunnableSequence > chain:load_memory_variables] s] Exiting Chain run w